сегодня мы поговорим о синтаксических анализаторах: посмотрим на их функции в деталях на примере UDPipe, но в конце тетрадки вспомним про другие(обзорно) и попробуем на парктике

## UDPipe

UDPipe - это готовый пайплайн* для токенизации, частеречной разметки, лемматизации и синтаксической разметки. Работает с файлами в формате [CoNLL-U](https://universaldependencies.org/format.html#syntactic-annotation)
<br>
Есть как готовые [предобученные модели](https://github.com/jwijffels/udpipe.models.ud.2.0/tree/master/inst/udpipe-ud-2.0-170801), так и возможность обучить модель на своих данных (но это небыстро). 

Использует теги [UD-формата](https://universaldependencies.org/)
______
*пайплайном называют цепочку из нескольких инструментов, используемых один за другим (pipeline, от англ. pipe- труба)*

поля в CoNLL-U формате:

* ID: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0)
* FORM: Word form or punctuation symbol.
* LEMMA: Lemma or stem of word form.
* UPOS: Universal part-of-speech tag.
* XPOS: Language-specific part-of-speech tag; underscore if not available.
* FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
* HEAD: Head of the current word, which is either a value of ID or zero (0).
* DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
* DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
* MISC: Any other annotation.

Так выглядит текст в CoNLL-U формате

    1	Мэр	_	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing|fPOS=NOUN++	2	nsubj	_	_
    2	открыл	_	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act|fPOS=VERB++	0	ROOT	_	_
    3	новый	_	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing|fPOS=ADJ++	4	amod	_	_
    4	парк	_	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing|fPOS=NOUN++	2	dobj	_	_
    5	и	_	CONJ	_	fPOS=CONJ++	4	cc	_	_
    6	детскую	_	ADJ	_	Case=Acc|Degree=Pos|Gender=Fem|Number=Sing|fPOS=ADJ++	7	amod	_	_
    7	площадку	_	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing|fPOS=NOUN++	4	conj	_	_
    8	.	_	PUNCT	.	fPOS=PUNCT++.	2	punct	_	_

**всякое про UDPipe**
* [общая информация](http://ufal.mff.cuni.cz/udpipe#language_models)
* [мануал](http://ufal.mff.cuni.cz/udpipe/users-manual)
* [репозиторий](https://github.com/ufal/udpipe)
* [статья с описанием архитектуры](http://ufal.mff.cuni.cz/~straka/papers/2017-conll_udpipe.pdf)
* [онлайн-версия](http://lindat.mff.cuni.cz/services/udpipe/)



### шаг1: установка библиотеки
полная информация о разных способах установки UDPipe [здесь](http://ufal.mff.cuni.cz/udpipe/install)
<br>
Мы попробуем простую, установив UPpipe как специальную библиотеку для питона, через ```pip3```

[документация для питонской библиотеки](https://pypi.org/project/ufal.udpipe/)

In [1]:
pip install spacy-udpipe

Note: you may need to restart the kernel to use updated packages.


In [1]:
import spacy_udpipe

spacy_udpipe.download("ru") # download English model

text = "Мэр открыл новый парк и детскую площадку."
nlp = spacy_udpipe.load("ru")

doc = nlp(text)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_)

Already downloaded a model for the 'ru' language
Мэр мэр NOUN nsubj
открыл открыть VERB ROOT
новый новый ADJ amod
парк парк NOUN obj
и и CCONJ cc
детскую детский ADJ amod
площадку площадка NOUN conj
. . PUNCT punct


In [497]:
# ячейка может выполняться долго, это нормально
! pip3 install ufal.udpipe
from ufal.udpipe import Model, Pipeline #импортируем нужные части из модуля

### шаг2: выбор модели

Чтобы работать с UDPipe, нужно выбрать модель: уже готовую, или обучить на своих данных.
<br>
Мы сегодня не будем обучать новые модели (это слишком долго), а используем [готовую модель для русского](https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe)

In [2]:
# сохраняем в переменную UDPIPE_MODEL_FN модель, которую загружаем из репозитория UD 
UDPIPE_MODEL_FN = "model_ru.udpipe"
!wget -O {UDPIPE_MODEL_FN} https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true

--2021-02-12 18:09:06--  https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe [following]
--2021-02-12 18:09:06--  https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe [following]
--2021-02-12 18:09:06--  https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe
Re

  7900K .......... .......... .......... .......... .......... 61% 40,3M 1s
  7950K .......... .......... .......... .......... .......... 61% 6,80M 1s
  8000K .......... .......... .......... .......... .......... 62% 33,4M 1s
  8050K .......... .......... .......... .......... .......... 62% 31,2M 1s
  8100K .......... .......... .......... .......... .......... 62% 2,11M 1s
  8150K .......... .......... .......... .......... .......... 63% 39,3M 1s
  8200K .......... .......... .......... .......... .......... 63% 43,2M 1s
  8250K .......... .......... .......... .......... .......... 64% 38,8M 1s
  8300K .......... .......... .......... .......... .......... 64% 50,4M 1s
  8350K .......... .......... .......... .......... .......... 64% 9,38M 1s
  8400K .......... .......... .......... .......... .......... 65% 17,2M 1s
  8450K .......... .......... .......... .......... .......... 65% 11,3M 1s
  8500K .......... .......... .......... .......... .......... 66% 9,86M 1s
  8550K ....

In [498]:
UDPIPE_MODEL_FN = "model_ru.udpipe"
!wget -O {UDPIPE_MODEL_FN} https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-syntagrus-ud-2.0-170801.udpipe?raw=true

--2021-02-17 16:23:04--  https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-syntagrus-ud-2.0-170801.udpipe?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-syntagrus-ud-2.0-170801.udpipe [following]
--2021-02-17 16:23:04--  https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-syntagrus-ud-2.0-170801.udpipe
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-syntagrus-ud-2.0-170801.udpipe [following]
--2021-02-17 16:23:04--  https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.

  4850K .......... .......... .......... .......... .......... 11% 5,39M 10s
  4900K .......... .......... .......... .......... .......... 11% 5,51M 10s
  4950K .......... .......... .......... .......... .......... 11% 3,35M 10s
  5000K .......... .......... .......... .......... .......... 11% 6,17M 10s
  5050K .......... .......... .......... .......... .......... 11% 4,82M 10s
  5100K .......... .......... .......... .......... .......... 11% 6,23M 10s
  5150K .......... .......... .......... .......... .......... 11% 5,59M 9s
  5200K .......... .......... .......... .......... .......... 11% 3,77M 9s
  5250K .......... .......... .......... .......... .......... 12% 5,94M 9s
  5300K .......... .......... .......... .......... .......... 12% 4,94M 9s
  5350K .......... .......... .......... .......... .......... 12% 4,59M 9s
  5400K .......... .......... .......... .......... .......... 12% 4,56M 9s
  5450K .......... .......... .......... .......... .......... 12% 5,01M 9s
  5500

In [159]:
UDPIPE_MODEL_FN = "model_ru.udpipe"
!wget -O {UDPIPE_MODEL_FN} https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/english-ud-2.0-170801.udpipe?raw=true

--2021-02-17 14:37:26--  https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/english-ud-2.0-170801.udpipe?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/english-ud-2.0-170801.udpipe [following]
--2021-02-17 14:37:26--  https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/english-ud-2.0-170801.udpipe
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/english-ud-2.0-170801.udpipe [following]
--2021-02-17 14:37:27--  https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/english-ud-2.0-170801.udpipe
Re

 10350K .......... .......... .......... .......... .......... 63% 10,5M 2s
 10400K .......... .......... .......... .......... .......... 63% 10,9M 1s
 10450K .......... .......... .......... .......... .......... 64% 9,12M 1s
 10500K .......... .......... .......... .......... .......... 64% 16,0M 1s
 10550K .......... .......... .......... .......... .......... 64% 11,8M 1s
 10600K .......... .......... .......... .......... .......... 65% 8,69M 1s
 10650K .......... .......... .......... .......... .......... 65% 7,17M 1s
 10700K .......... .......... .......... .......... .......... 65% 10,7M 1s
 10750K .......... .......... .......... .......... .......... 66% 10,9M 1s
 10800K .......... .......... .......... .......... .......... 66% 10,5M 1s
 10850K .......... .......... .......... .......... .......... 66% 11,2M 1s
 10900K .......... .......... .......... .......... .......... 66% 11,0M 1s
 10950K .......... .......... .......... .......... .......... 67% 11,6M 1s
 11000K ....

In [499]:
model = Model.load(UDPIPE_MODEL_FN) # загружаем модель, сохраняем в переменную model

In [7]:
print(model)

<Swig Object of type 'model *' at 0x0000023E1A2B8FB0>


### шаг3: текст

In [4]:
sent = "Мэр открыл новый парк и детскую площадку."
print(sent)

Мэр открыл новый парк и детскую площадку.


### шаг4: анализируем

In [5]:
pipeline = Pipeline(model, 'generic_tokenizer', '','','')
#функции нужно 5 аргументов,но нам важны только 2
#сохраняем в переменную результат токенизации

ud_res = []
parsed = pipeline.process(sent) # функция process сделает синтаксический анализ, сохраняем еще раз

print(type(parsed), "\n\n", parsed) # печатаем результат


<class 'str'> 

 # newdoc
# newpar
# sent_id = 1
# text = Мэр открыл новый парк и детскую площадку.
1	Мэр	МЭР	NOUN	NN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	2	nsubj	_	_
2	открыл	ОТКРЫТЬ	VERB	VBC	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin	0	root	_	_
3	новый	НОВЫЙ	ADJ	JJL	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	4	amod	_	_
4	парк	ПАРК	NOUN	NN	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	2	obj	_	_
5	и	И	CCONJ	CC	_	7	cc	_	_
6	детскую	детскую	ADJ	JJL	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	7	amod	_	_
7	площадку	ПЛОЩАДКА	NOUN	NN	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	4	conj	_	SpaceAfter=No
8	.	.	PUNCT	.	_	2	punct	_	SpacesAfter=\n




дефолтный теггинг иногда может быть ошибочным: теггер предсказывает морфологические свойства токена по последним четырем символам каждого слова. Он генерирует гипотезы относительно части речи и морфологических тегов этого слова, а затем отбирает лучший вариант. 
<br>
*(можно улучшать качество, например, используя сторонний морфоанализатор и токенизатор, но это тема для отдельной пары, сегодня мы попробуем "ванильный" UDPipe)*

# способы применения
Зачем нужен синтаксический парсинг? 

Например, с помощью него можно находить в предложениях тройки субъект-глагол-объект, которые можно использовать для извлечения информации из текста.

**визуализация**

В nltk есть DependencyGraph, который умеет рисовать деревья (и ещё многое другое). Для того, чтобы визуализация работала корректно, ему нужна зависимость: graphviz.

In [ ]:
! pip3 install graphviz
! pip3 install pydot 
# ! brew install graphviz

Бибилиотеку grapghviz нужно поставить не только через pip, но и добавить в систему ([см этот тред](https://stackoverflow.com/questions/35064304/runtimeerror-make-sure-the-graphviz-executables-are-on-your-systems-path-aft))

Если возникает ошибка, попробуйте следующие команды:
* (Win) запустите в ячейке код  
```import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/' ```

* (Mac) в терминале: ```brew install graphviz ``` (проверьте, что у Вас стоит [homebrew](https://brew.sh/))

* (Linux) в терминале: ```sudo apt-get install graphviz```

In [ ]:
import os

In [ ]:
os.environ["PATH"] += os.pathsep + 'H:/Anaconda/Library/bin/graphviz/'

In [6]:
from nltk import DependencyGraph, Tree 
# вызываем классы, которые нарисуют нам деревья зависимостей

Для построения дерева нам нужно превратить файл в conllu-формате в список
<br> 
Еще нужно сделать тег ROOT в верхнем регистре, иначе он не находится

Создадим функцию, которую попробуем на результате UDPipe

In [7]:
def conllu_to_list(parser_result): 
# аргумент - это conllu-файл, который получили в результате синтаксического анализа 
    sents = []
    for sent in parser_result.split('\n\n'):
        # убираем коменты
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        # заменяем регистр для root
        sent = sent.replace('\troot\t', '\tROOT\t')
        sents.append(sent)
    return sents

## для UDPipe

In [8]:
# вызовем для результата UDPipe:

ud = conllu_to_list(parsed) # превратили в лист

print(type(ud), "\n\n", ud)

<class 'list'> 

 ['1\tМэр\tМЭР\tNOUN\tNN\tAnimacy=Anim|Case=Nom|Gender=Masc|Number=Sing\t2\tnsubj\t_\t_\n2\tоткрыл\tОТКРЫТЬ\tVERB\tVBC\tAspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin\t0\tROOT\t_\t_\n3\tновый\tНОВЫЙ\tADJ\tJJL\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Sing\t4\tamod\t_\t_\n4\tпарк\tПАРК\tNOUN\tNN\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Sing\t2\tobj\t_\t_\n5\tи\tИ\tCCONJ\tCC\t_\t7\tcc\t_\t_\n6\tдетскую\tдетскую\tADJ\tJJL\tAnimacy=Inan|Case=Acc|Gender=Fem|Number=Sing\t7\tamod\t_\t_\n7\tплощадку\tПЛОЩАДКА\tNOUN\tNN\tAnimacy=Inan|Case=Acc|Gender=Fem|Number=Sing\t4\tconj\t_\tSpaceAfter=No\n8\t.\t.\tPUNCT\t.\t_\t2\tpunct\t_\tSpacesAfter=\\n', '']


In [9]:
ud_graph = DependencyGraph(tree_str=ud[0])

In [ ]:
ud_graph

In [ ]:
# нарисуем граф для UDPipe
for elem in ud:
    display(DependencyGraph(tree_str=elem))
# ud_graph = DependencyGraph(tree_str=ud[x])
# # print(ud_graph)
# ud_graph

а вот еще один способ визуализации дерева

In [ ]:
ud_tree = ud_graph.tree()

print(ud_tree.pretty_print())

In [ ]:
# посмотрим, на что вообще можно разбить предложение
list(ud_graph.triples())

**Тройки глагол-объект-субьект:**

Предположим, нам нужно вытащить только ту тройку, которая расскажет о предикате (сказуемом), субъекте (подлежащем) и объекте (дополнении).

In [ ]:
def get_sov(text): # зададим функцию, которая будет вытаскивать нужную тройку
    
    graph = DependencyGraph(tree_str=ud[0])
    #ud_graph = DependencyGraph(tree_str=ud[0]) 
    sov = {} # пустой словарь, будем в него складывать ключи и значения
    
    for triple in graph.triples(): # для каждого триплета из всех
        if triple[0][1] == 'VERB': # если тег первого элемента - VERB
            sov[triple[0][0]] = {'subj':'','obj':''}
            
    for triple in graph.triples(): # почему
        
        if triple[1] == 'nsubj': # если второй элемент -- существительное
            if triple[0][1] == 'VERB': # при этом если тег первого элемента - VERB
                sov[triple[0][0]]['subj']  = triple[2][0] # сохраняем значение 
        elif triple[1] == 'obj': # соответственно для дополнения
            if triple[0][1] == 'VERB':
                sov[triple[0][0]]['obj'] = triple[2][0]
    return sov

 проверим:

In [ ]:
sov = get_sov(sent)
print("\n",sov)

In [ ]:
# попробуем разное расположение подлежащего и сказуемого
test_sent = ["Мэр открыл площадку",
             "Открыл мэр площадку",
             "Площадку открыл мэр"]

In [ ]:
for elem in test_sent:
    sov = get_sov(elem)
    print("\n",sov)

улучшим функцию, теперь она находит однородные дополнения *(парк и площадку)*

Нам понадобится defaultdict -- это контейнер, похожий на словарь, который при доступе по несуществующему ключу присваивает ему заданное(дефолтное) значение
[подробнее](https://docs.python.org/3/library/collections.html#collections.defaultdict)

проверим:

In [ ]:
from collections import defaultdict as dd

def get_sov(sent):
    graph = DependencyGraph(tree_str=ud[0])

    subjects = dd(lambda : {"subject": "", "verb": "", "objects": []})
    # создаем словарь для подлежащих, в нем есть ключи но пока нулевые значения
    
    verbs = dd(lambda : {"subject": "", "verb": "", "objects": []})
    # такой же словарь для сказуемых
    
    for triple in graph.triples(): # проверяем все тройки
        if triple[1] == 'conj': # если второй элемент - союз 
            subjects[triple[0][0]]["objects"].append(triple[2][0])
            # сохраняем значение в раздел "дополнения"
                
        # аналогично для существительных и глаголов
        if triple[1] == 'nsubj': 
            if triple[0][1] == 'VERB':
                verbs[triple[0][0]]["subject"] = triple[2][0]
        if triple[1] == 'obj':
            if triple[0][1] == 'VERB':
                subjects[triple[2][0]]["verb"] = triple[0][0]
                subjects[triple[2][0]]["objects"].append(triple[2][0])

    
    sovs = [] # пустой список, сюда будем складывать нужные фрагменты всего что нашли

    print(subjects, verbs) # печатаем все содержимое словарей (можно закомментить)
    
    for v in subjects.values(): # для каждого значения в словаре подлежащих
        for obj in v["objects"]: # для каждого значения в списке дополнений
            sovs.append((verbs[v["verb"]]["subject"], v["verb"], obj))
            # добавляем в список соответственно
    return sovs # возвращаем готовый список

In [ ]:
sov = get_sov(sent)
print("\n",sov)

## идеи дальше:
(на выбор)
1. Выбрать текст (на русском), с которым было бы интересно поработать. Попробовать вытащить все тройки предикат-субъект-объект из выбранного текста, посмотреть на результат, проинтерпретировать
2. Написать функцию,которая бы учитывала сложные сказуемые*(начинает уходить, устал думать)* как единый элемент (она похожа на функцию, которая собирает однородные дополнения, нам также нужно что-то сделать с тегами)
3. попробовать предобученные модели для других языков
4. Попробовать парсеры на предложениях с [синтаксической омонимией](https://github.com/sjut/DPO_Materials/blob/ff1341f1d82ca11a763e15d76601bd6898958323/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8F/%D1%81%D0%B8%D0%BD%D1%82_%D0%BD%D0%B5%D0%BE%D0%B4%D0%BD%D0%BE%D0%B7%D0%BD%D0%B0%D1%87%D0%BD%D0%BE%D1%81%D1%82%D1%8C.txt), проинтерпретировать результаты 
(для этого текст, который вы хотите анализировать, необходимо сохранить в переменную одним из удобных способов)

# что есть еще? 

более немейнстримные парсеры

## DeepPavlov

[ссылка на тетрадку с подробностями](https://github.com/nstsj/python_for_CL/blob/master/syntax_parsing/syntax_analysis_DeepPavlov.ipynb)


## SyntaxNet

SyntaxNet — это гугловская библиотека для определения синтаксических связей (использует нейронную сеть на TensorFlow). Поддерживает 40 языков, в том числе  русский.

* [документация](https://github.com/tensorflow/models/blob/master/research/syntaxnet/README.md)
* [репозиторий на гитхабе](https://github.com/tensorflow/models/tree/master/research/syntaxnet)
* [тьюториал](https://github.com/tensorflow/models/blob/master/research/syntaxnet/g3doc/syntaxnet-tutorial.md)
* [ссылка на обученные модели](https://github.com/tensorflow/models/blob/master/research/syntaxnet/g3doc/universal.md)

## Stanford Parser

* [о проекте](https://nlp.stanford.edu/software/stanford-dependencies.shtml#Languages)
* [о парсере](https://nlp.stanford.edu/software/lex-parser.html)
* [питонская обертка](http://projects.csail.mit.edu/spatial/Stanford_Parser)

## SpaCy

библиотека для различных NLP-задач от (токенизации до NER и др), в том числе умеет делать [синтаксический анализ предложения в грамматике зависимостей](https://spacy.io/usage/linguistic-features#dependency-parse)

In [ ]:
# Собираем сложные сказуемые:

In [7]:
sent = "Он надеялся подстрекнуть его великодушие и склонить его на свою сторону." 
print(sent)

Он надеялся подстрекнуть его великодушие и склонить его на свою сторону.


In [8]:
pipeline = Pipeline(model, 'generic_tokenizer', '','','')

ud_res = []
parsed = pipeline.process(sent)

print(type(parsed), "\n\n", parsed)


<class 'str'> 

 # newdoc
# newpar
# sent_id = 1
# text = Он надеялся подстрекнуть его великодушие и склонить его на свою сторону.
1	Он	он	PRON	_	Case=Nom|Gender=Masc|Number=Sing|Person=3	2	nsubj	_	_
2	надеялся	надеяться	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid	0	root	_	_
3	подстрекнуть	подстрекать	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	2	xcomp	_	_
4	его	он	PRON	_	Case=Gen|Gender=Masc|Number=Sing|Person=3	5	nmod	_	_
5	великодушие	великодуший	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	3	obj	_	_
6	и	и	CCONJ	_	_	7	cc	_	_
7	склонить	склонять	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	3	conj	_	_
8	его	он	PRON	_	Case=Acc|Gender=Masc|Number=Sing|Person=3	7	obj	_	_
9	на	на	ADP	_	_	11	case	_	_
10	свою	свой	DET	_	Case=Acc|Gender=Fem|Number=Sing	11	amod	_	_
11	сторону	сторона	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	7	obl	_	SpaceAfter=No
12	.	.	PUNCT	_	_	11	punct	_	SpacesAfter=\n




In [9]:
from nltk import DependencyGraph, Tree

In [10]:
def conllu_to_list(parser_result): 
    sents = []
    for sent in parser_result.split('\n\n'):
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        sent = sent.replace('\troot\t', '\tROOT\t')
        sents.append(sent)
    return sents

In [11]:
ud = conllu_to_list(parsed)

print(type(ud), "\n\n", ud)

<class 'list'> 

 ['1\tОн\tон\tPRON\t_\tCase=Nom|Gender=Masc|Number=Sing|Person=3\t2\tnsubj\t_\t_\n2\tнадеялся\tнадеяться\tVERB\t_\tAspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid\t0\tROOT\t_\t_\n3\tподстрекнуть\tподстрекать\tVERB\t_\tAspect=Perf|VerbForm=Inf|Voice=Act\t2\txcomp\t_\t_\n4\tего\tон\tPRON\t_\tCase=Gen|Gender=Masc|Number=Sing|Person=3\t5\tnmod\t_\t_\n5\tвеликодушие\tвеликодуший\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Neut|Number=Sing\t3\tobj\t_\t_\n6\tи\tи\tCCONJ\t_\t_\t7\tcc\t_\t_\n7\tсклонить\tсклонять\tVERB\t_\tAspect=Perf|VerbForm=Inf|Voice=Act\t3\tconj\t_\t_\n8\tего\tон\tPRON\t_\tCase=Acc|Gender=Masc|Number=Sing|Person=3\t7\tobj\t_\t_\n9\tна\tна\tADP\t_\t_\t11\tcase\t_\t_\n10\tсвою\tсвой\tDET\t_\tCase=Acc|Gender=Fem|Number=Sing\t11\tamod\t_\t_\n11\tсторону\tсторона\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Fem|Number=Sing\t7\tobl\t_\tSpaceAfter=No\n12\t.\t.\tPUNCT\t_\t_\t11\tpunct\t_\tSpacesAfter=\\n', '']


In [12]:
ud_graph = DependencyGraph(tree_str=ud[0])

In [13]:
ud_tree = ud_graph.tree()

print(ud_tree.pretty_print())

      надеялся                                       
  _______|_______________                             
 |                  подстрекнуть                     
 |        _______________|__________                  
 |       |                       склонить            
 |       |        __________________|________         
 |  великодушие  |       |                сторону    
 |       |       |       |           ________|_____   
 Он     его      и      его         на      свою   . 

None


In [14]:
list(ud_graph.triples())

[(('надеялся', 'VERB'), 'nsubj', ('Он', 'PRON')),
 (('надеялся', 'VERB'), 'xcomp', ('подстрекнуть', 'VERB')),
 (('подстрекнуть', 'VERB'), 'obj', ('великодушие', 'NOUN')),
 (('великодушие', 'NOUN'), 'nmod', ('его', 'PRON')),
 (('подстрекнуть', 'VERB'), 'conj', ('склонить', 'VERB')),
 (('склонить', 'VERB'), 'cc', ('и', 'CCONJ')),
 (('склонить', 'VERB'), 'obj', ('его', 'PRON')),
 (('склонить', 'VERB'), 'obl', ('сторону', 'NOUN')),
 (('сторону', 'NOUN'), 'case', ('на', 'ADP')),
 (('сторону', 'NOUN'), 'amod', ('свою', 'DET')),
 (('сторону', 'NOUN'), 'punct', ('.', 'PUNCT'))]

In [15]:
from collections import defaultdict as dd

def get_sov(sent):
    graph = DependencyGraph(tree_str=ud[0])

    subjects = dd(lambda : {"subject": "", "verb": "", "infinitives": []})
        
    verbs = dd(lambda : {"subject": "", "verb": "", "infinitives": []})
        
    for triple in graph.triples(): 
        if triple[1] == 'conj': 
            subjects[triple[0][0]]["infinitives"].append(triple[2][0])
       
        if triple[1] == 'nsubj': 
            if triple[0][1] == 'VERB':
                verbs[triple[0][0]]["subject"] = triple[2][0]
        if triple[1] == 'xcomp':
            if triple[0][1] == 'VERB':
                subjects[triple[2][0]]["verb"] = triple[0][0]
                subjects[triple[2][0]]["infinitives"].append(triple[2][0])

    
    sovs = [] 

    print(subjects, verbs)
    
    for v in subjects.values(): 
        for inf in v["infinitives"]: 
            sovs.append((verbs[v["verb"]]["subject"], v["verb"], inf))
            
    return sovs

In [16]:
sov = get_sov(sent)
print("\n",sov)

defaultdict(<function get_sov.<locals>.<lambda> at 0x000001771E314C10>, {'подстрекнуть': {'subject': '', 'verb': 'надеялся', 'infinitives': ['подстрекнуть', 'склонить']}}) defaultdict(<function get_sov.<locals>.<lambda> at 0x000001771E314B80>, {'надеялся': {'subject': 'Он', 'verb': '', 'infinitives': []}})

 [('Он', 'надеялся', 'подстрекнуть'), ('Он', 'надеялся', 'склонить')]


In [599]:
text = """Он из Германии туманной привез учености плоды.
Эти типы стали есть на складе.
Портрет из кости Екатерины Второй.
Пете звонить нельзя.
Нужно будет переизбрать заместителя.
Парламент защищает правительство.
Фонд социальной защиты Самарского района.
Дом культуры лакокрасочного завода имени М.В.Ломоносова.
Я люблю многозначность сильнее, чем большинство людей.
Чужим телефоном пользоваться нельзя."""

In [600]:
pipeline = Pipeline(model, 'generic_tokenizer', '','','')

ud_res = []
parsed1 = pipeline.process(text) 
print(type(parsed1), "\n\n", parsed1)

<class 'str'> 

 # newdoc
# newpar
# sent_id = 1
# text = Он из Германии туманной привез учености плоды.
1	Он	он	PRON	_	Case=Nom|Gender=Masc|Number=Sing|Person=3	5	nsubj	_	_
2	из	из	ADP	_	_	3	case	_	_
3	Германии	германия	PROPN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	5	obl	_	_
4	туманной	туманный	ADJ	_	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	3	amod	_	_
5	привез	привозить	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
6	учености	ученость	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	5	obl	_	_
7	плоды	плод	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur	6	appos	_	SpaceAfter=No
8	.	.	PUNCT	_	_	7	punct	_	SpacesAfter=\n

# sent_id = 2
# text = Эти типы стали есть на складе.
1	Эти	этот	DET	_	Case=Nom|Number=Plur	2	amod	_	_
2	типы	тип	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur	3	nsubj	_	_
3	стали	стать	VERB	_	Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
4	есть	есть	VERB	_	Aspect=Im

In [601]:
ud1 = conllu_to_list(parsed1)

print(type(ud1), "\n\n", ud1)

<class 'list'> 

 ['1\tОн\tон\tPRON\t_\tCase=Nom|Gender=Masc|Number=Sing|Person=3\t5\tnsubj\t_\t_\n2\tиз\tиз\tADP\t_\t_\t3\tcase\t_\t_\n3\tГермании\tгермания\tPROPN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Sing\t5\tobl\t_\t_\n4\tтуманной\tтуманный\tADJ\t_\tCase=Gen|Degree=Pos|Gender=Fem|Number=Sing\t3\tamod\t_\t_\n5\tпривез\tпривозить\tVERB\t_\tAspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act\t0\tROOT\t_\t_\n6\tучености\tученость\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Sing\t5\tobl\t_\t_\n7\tплоды\tплод\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Plur\t6\tappos\t_\tSpaceAfter=No\n8\t.\t.\tPUNCT\t_\t_\t7\tpunct\t_\tSpacesAfter=\\n', '1\tЭти\tэтот\tDET\t_\tCase=Nom|Number=Plur\t2\tamod\t_\t_\n2\tтипы\tтип\tNOUN\t_\tAnimacy=Inan|Case=Nom|Gender=Masc|Number=Plur\t3\tnsubj\t_\t_\n3\tстали\tстать\tVERB\t_\tAspect=Perf|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act\t0\tROOT\t_\t_\n4\tесть\tесть\tVERB\t_\tAspect=Imp|VerbForm=Inf|Voice=A

In [613]:
ud_graph1 = DependencyGraph(tree_str=ud1[9])

In [614]:
ud_tree1 = ud_graph1.tree()
print(ud_tree1.pretty_print())

    нельзя             
  ____|_________        
 |         пользоваться
 |              |       
 |          телефоном  
 |              |       
 .            Чужим    

None


In [610]:
# Он из Германии туманной привез учености плоды. - распозналось как из туманной Германии он привез плоды учености.
# Эти типы стали есть на складе. - "Стали" распозналось как глагол. "Есть" не распозналось как open clausal complement 
# (xcomp), т.е. как вторая часть составного глагольного сказуемого. А распозналось как связка (copula). 
# То есть "Эти типы стали" и "есть на складе" - это как бы две отдельные части предложения.
# Портрет из кости Екатерины Второй. - Второй(род - муж.) портрет, сделанный из кости Екатерины.
# Пете звонить нельзя. - "Петя" распозналось как глагол во множ. числе, 2-е лицo. "Звонить" распозналось как глагол, 
# нельзя - как наречие. В целом предложение не распозналось так, чтобы все его части можно было сложить вместе.
# Нужно будет переизбрать заместителя. В данном случае присутствует лексическая неоднозначность, то есть смысл слова 
# "переизбрать" (избрать председателя еще раз или выбрать на его место другого человека) не зависит от синтаксиса. 
# Поэтому предложение можно понять по-разному.
# Парламент защищает правительство. - "Парламент" и "правительство" распознались как сущ. в именит. падеже. 
# Но напротив правительства стоит obj, т.е. объект, на который направлено действие. Значит, предложение можно
# понять как парламент осуществляет защиту правительства.
# Фонд социальной защиты Самарского района. - Фонд социальной защиты жителей Самарского района.
# Дом культуры лакокрасочного завода имени М.В.Ломоносова. - Дом культуры лакокрасочного завода, названного 
# в честь М.В.Ломоносова.
# Я люблю многозначность сильнее, чем большинство людей. - Я люблю многозначность сильнее, чем ее любит 
# большинство (именит. падеж) людей. 
# Чужим телефоном пользоваться нельзя. - Нельзя пользоваться не своим телефоном.

In [486]:
text = """Squad helps dog bite victim.
Helicopter powered by human flies.
Flying planes can be dangerous.
I saw Grand Canyon flying to LA.
I made her duck.
Teacher strikes idle kids.
Milk drinkers are turning to powder.
Drunk gets nine months in violin case."""

In [487]:
pipeline = Pipeline(model, 'generic_tokenizer', '','','')

ud_res = []
parsed1 = pipeline.process(text) 
print(type(parsed1), "\n\n", parsed1) 

<class 'str'> 

 # newdoc
# newpar
# sent_id = 1
# text = Squad helps dog bite victim.
1	Squad	Squad	PROPN	NNP	Number=Sing	2	nsubj	_	_
2	helps	help	VERB	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	_
3	dog	dog	NOUN	NN	Number=Sing	5	compound	_	_
4	bite	bite	NOUN	NN	Number=Sing	5	compound	_	_
5	victim	victim	NOUN	NN	Number=Sing	2	obj	_	SpaceAfter=No
6	.	.	PUNCT	.	_	2	punct	_	SpacesAfter=\n

# sent_id = 2
# text = Helicopter powered by human flies.
1	Helicopter	helicopter	NOUN	NN	Number=Sing	0	root	_	_
2	powered	powere	VERB	VBN	Tense=Past|VerbForm=Part	1	acl	_	_
3	by	by	ADP	IN	_	5	case	_	_
4	human	human	ADJ	JJ	Degree=Pos	5	amod	_	_
5	flies	fly	NOUN	NNS	Number=Plur	2	obl	_	SpaceAfter=No
6	.	.	PUNCT	.	_	1	punct	_	SpacesAfter=\n

# sent_id = 3
# text = Flying planes can be dangerous.
1	Flying	Flying	VERB	VBG	VerbForm=Ger	2	amod	_	_
2	planes	plane	NOUN	NNS	Number=Plur	5	nsubj	_	_
3	can	can	AUX	MD	VerbForm=Fin	5	aux	_	_
4	be	be	AUX	VB	VerbForm=Inf	5	cop	_	_
5	dangerous	da

In [488]:
def conllu_to_list(parser_result): 
# аргумент - это conllu-файл, который получили в результате синтаксического анализа 
    sents = []
    for sent in parser_result.split('\n\n'):
        # убираем коменты
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        # заменяем регистр для root
        sent = sent.replace('\troot\t', '\tROOT\t')
        sents.append(sent)
    return sents

In [489]:
ud1 = conllu_to_list(parsed1)

print(type(ud1), "\n\n", ud1)

<class 'list'> 

 ['1\tSquad\tSquad\tPROPN\tNNP\tNumber=Sing\t2\tnsubj\t_\t_\n2\thelps\thelp\tVERB\tVBZ\tMood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin\t0\tROOT\t_\t_\n3\tdog\tdog\tNOUN\tNN\tNumber=Sing\t5\tcompound\t_\t_\n4\tbite\tbite\tNOUN\tNN\tNumber=Sing\t5\tcompound\t_\t_\n5\tvictim\tvictim\tNOUN\tNN\tNumber=Sing\t2\tobj\t_\tSpaceAfter=No\n6\t.\t.\tPUNCT\t.\t_\t2\tpunct\t_\tSpacesAfter=\\n', '1\tHelicopter\thelicopter\tNOUN\tNN\tNumber=Sing\t0\tROOT\t_\t_\n2\tpowered\tpowere\tVERB\tVBN\tTense=Past|VerbForm=Part\t1\tacl\t_\t_\n3\tby\tby\tADP\tIN\t_\t5\tcase\t_\t_\n4\thuman\thuman\tADJ\tJJ\tDegree=Pos\t5\tamod\t_\t_\n5\tflies\tfly\tNOUN\tNNS\tNumber=Plur\t2\tobl\t_\tSpaceAfter=No\n6\t.\t.\tPUNCT\t.\t_\t1\tpunct\t_\tSpacesAfter=\\n', '1\tFlying\tFlying\tVERB\tVBG\tVerbForm=Ger\t2\tamod\t_\t_\n2\tplanes\tplane\tNOUN\tNNS\tNumber=Plur\t5\tnsubj\t_\t_\n3\tcan\tcan\tAUX\tMD\tVerbForm=Fin\t5\taux\t_\t_\n4\tbe\tbe\tAUX\tVB\tVerbForm=Inf\t5\tcop\t_\t_\n5\tdangerous\tdangerous\tADJ\t

In [490]:
from nltk import DependencyGraph, Tree 

In [493]:
ud_graph1 = DependencyGraph(tree_str=ud1[7])

In [494]:
ud_tree1 = ud_graph1.tree()
print(ud_tree1.pretty_print())

          gets                   
   ________|_____                 
  |    |       months            
  |    |    _____|_____           
  |    |   |          case       
  |    |   |      _____|_____     
Drunk  .  nine   in        violin

None


In [478]:
# Squad helps dog bite victim. "Squad" распозналось как имя собственное. Squad помогает жертве укуса собаки. 
# Helicopter powered by human flies. Вертолет, управляемый человеческими мухами.
# Flying planes can be dangerous. Летающие (герундий) самолеты могут быть опасны.
# I saw Grand Canyon flying to LA. Я видел Большой каньон, летящий (герундий) в Лос-Анджелес. 
# I made her duck. Я сделал её утку.
# Teacher strikes idle kids. "Teacher" распозналось как детерменатив. "Strikes" распозналось как сущ. во множ. числе,
# "idle" - как прил., "kids" - сущ. во множ. числе. Т.е. "idle kids" можно перевести как "ленивые дети". 
# Strikes - удары или забастовки. В целом предложение не распозналось так, чтобы все его части можно было сложить вместе.
# Milk drinkers are turning to powder. Пьющие молоко превращаются в порошок (сущ.)/переходят на (молочный) порошок. 
# Смысл слова "turning" не зависит от синтаксиса, поэтому предложение остается неоднозначиным. 
# Drunk gets nine months in violin case. Пьяница получает девять месяцев в скрипичном ящике.
# (violin case зависит от nine months)